<h1>Part 1 - Creation of Canada's Postal Codes DataFrame</h1>

<h2>Scraping Wikipedia</h2>

In [4]:
import pandas as pd
import numpy as np

In [5]:
html_content = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print(html_content)

[    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25          M8C      Not assigned   


In [6]:
pc_can=pd.DataFrame(html_content[0])
pc_can.rename(columns={"Postal Code":'PostalCode'}, inplace=True)
pc_can.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
pc_can.replace("Not assigned", np.nan, inplace = True)
pc_can.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Cheking for missing values</h3>

In [8]:
missing_data = pc_can.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

PostalCode
False    180
Name: PostalCode, dtype: int64

Borough
False    103
True      77
Name: Borough, dtype: int64

Neighborhood
False    103
True      77
Name: Neighborhood, dtype: int64



In [9]:
pc_can.dropna(subset=["Borough"],axis=0, inplace=True)
pc_can.reset_index(drop=True, inplace=True)
pc_can.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
pc_can.shape

(103, 3)

<h1>PART 2 - Latitude and longitude coordinates of each neighborhood</h1>

In [12]:
import geocoder

In [13]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords

In [14]:
postal_codes = pc_can['PostalCode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

pc_can_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
pc_can['Latitude'] = pc_can_coords['Latitude']
pc_can['Longitude'] = pc_can_coords['Longitude']

pc_can.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


<h1>Part 3. Exploring and clustering the neighborhoods in Toronto</h1>

In [42]:
import folium

from geopy.geocoders import Nominatim

import json

import requests

from pandas.io.json import json_normalize

In [16]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [43]:
df_tor=pc_can[pc_can['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_tor.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
4,M4E,East Toronto,The Beaches,43.67703,-79.29542


In [44]:
# create map of New York using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [45]:
CLIENT_ID = 'SHE2VTZE10WC04OEHVTAHUIYRMA3VBQHYCKSHLEP0CNWW5YJ' # your Foursquare ID
CLIENT_SECRET = 'M4VVFHKYJER5XZUDJVQ4KDLUXKKRSNJQXO5QPMKVV1INMEPQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SHE2VTZE10WC04OEHVTAHUIYRMA3VBQHYCKSHLEP0CNWW5YJ
CLIENT_SECRET:M4VVFHKYJER5XZUDJVQ4KDLUXKKRSNJQXO5QPMKVV1INMEPQ


In [46]:
def getNearbyVenues(names, latitudes,longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude'],
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [48]:
print(toronto_venues.shape)
toronto_venues.head()

(1753, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [49]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,67,67,67,67,67,67
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",72,72,72,72,72,72
Central Bay Street,69,69,69,69,69,69
Christie,10,10,10,10,10,10
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


In [51]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(5)

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.014925,0.000000,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,...,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.011905,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.023810,...,0.00,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.000000,0.010000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.013889,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.000000,0.013889,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.014493,0.014493,0.000000,...,0.00,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.000000,0.000000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.012987,0.000000,0.000000,0.012987,0.00,0.000000,0.000000,0.000000,0.012987,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.040000,0.00,0.000000,0.010000,0.000000,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [53]:
toronto_grouped.shape

(39, 228)

In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.04
2          Restaurant  0.04
3  Seafood Restaurant  0.03
4  Italian Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2          Bar  0.06
3   Restaurant  0.05
4    Gift Shop  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
             venue  freq
0      Coffee Shop  0.09
1            Hotel  0.05
2             Café  0.04
3       Taco Place  0.02
4  Thai Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2                Café  0.04
3                 Bar  0.04
4           Speakeasy  0.03


----Central Bay Street----
               venue  freq
0        Coffee Shop  0.10
1     Clothing Sto

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pharmacy,Lounge,Seafood Restaurant,Cheese Shop,Breakfast Spot,Beer Bar,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Coffee Shop,Restaurant,Sandwich Place,Gift Shop,Supermarket,Furniture / Home Store,Nightclub,Burrito Place
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Burrito Place,Sandwich Place,Bookstore,Concert Hall,Mediterranean Restaurant,Steakhouse,Bar
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,Bar,Sandwich Place,Electronics Store,Park,Bakery,Speakeasy,Pizza Place
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Pizza Place,Plaza,Electronics Store,Sandwich Place,Bubble Tea Shop


<h3>Clustering Neighborhoods</h3>

In [59]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,0,Coffee Shop,Breakfast Spot,Thai Restaurant,Health Food Store,Pub,Distribution Center,Restaurant,Electronics Store,Event Space,Food Truck
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,0,Coffee Shop,Café,Sandwich Place,Park,Pizza Place,Chinese Restaurant,Salon / Barbershop,Clothing Store,Restaurant,Pub
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,0,Coffee Shop,Clothing Store,Hotel,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Theater,Ramen Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557,0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Clothing Store,American Restaurant,Cosmetics Shop,Cocktail Bar,Lingerie Store,Farmers Market
4,M4E,East Toronto,The Beaches,43.67703,-79.29542,0,Health Food Store,Pub,Asian Restaurant,Trail,Women's Store,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [70]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

<h4>Cluster 1</h4>

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Breakfast Spot,Thai Restaurant,Health Food Store,Pub,Distribution Center,Restaurant,Electronics Store,Event Space,Food Truck
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Café,Sandwich Place,Park,Pizza Place,Chinese Restaurant,Salon / Barbershop,Clothing Store,Restaurant,Pub
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Hotel,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Theater,Ramen Restaurant,Bookstore
3,St. James Town,0,Seafood Restaurant,Café,Restaurant,Coffee Shop,Clothing Store,American Restaurant,Cosmetics Shop,Cocktail Bar,Lingerie Store,Farmers Market
4,The Beaches,0,Health Food Store,Pub,Asian Restaurant,Trail,Women's Store,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
5,Berczy Park,0,Coffee Shop,Restaurant,Cocktail Bar,Pharmacy,Lounge,Seafood Restaurant,Cheese Shop,Breakfast Spot,Beer Bar,Farmers Market
6,Central Bay Street,0,Coffee Shop,Clothing Store,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Pizza Place,Plaza,Electronics Store,Sandwich Place,Bubble Tea Shop
7,Christie,0,Café,Grocery Store,Candy Store,Baby Store,Italian Restaurant,Playground,Coffee Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
8,"Richmond, Adelaide, King",0,Café,Hotel,Coffee Shop,Restaurant,Gym,Japanese Restaurant,Salad Place,American Restaurant,Asian Restaurant,Steakhouse
9,"Dufferin, Dovercourt Village",0,Park,Furniture / Home Store,Pizza Place,Bakery,Bus Line,Middle Eastern Restaurant,Pharmacy,Brazilian Restaurant,Café,Bar


<h4>Cluster 2</h4>

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",1,Park,Women's Store,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


<h4>Cluster 3</h4>

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,"Summerhill West, Rathnelly, South Hill, Forest...",2,Coffee Shop,Light Rail Station,Supermarket,Liquor Store,Women's Store,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


<h4>Cluster 4</h4>

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,3,Home Service,Health & Beauty Service,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


<h4>Cluster 5</h4>

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,4,Photography Studio,Park,Electronics Store,Women's Store,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
20,Davisville North,4,Food & Drink Shop,Park,Hotel,Breakfast Spot,Department Store,Gym / Fitness Center,Gym,Event Space,Falafel Restaurant,Farm
22,"High Park, The Junction South",4,Convenience Store,Park,Residential Building (Apartment / Condo),Sandwich Place,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
23,"North Toronto West, Lawrence Park",4,Playground,Gym Pool,Park,Garden,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
29,"Moore Park, Summerhill East",4,Tennis Court,Gym,Trail,Park,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
33,Rosedale,4,Playground,Park,Shop & Service,Bike Trail,Tennis Court,Dance Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
